In [111]:
from bokeh.plotting import *
from bokeh.layouts import column
from bokeh.transform import jitter, factor_cmap
from bokeh.models import ColumnDataSource, CustomJS, Select, RadioButtonGroup, Div, HoverTool, FactorRange
from bokeh.palettes import Pastel2
import pandas as pd
output_notebook()

Loading BokehJS ...

In [10]:
df = pd.read_csv('all_data_summary.csv')
df.head()

,Unnamed: 0,filename,cell_type,cell_type_dual,mesh_type,activation_time,cell_surface_area_um^2,mesh_density,equivalent_diameter_area_um^2_mean,equivalent_diameter_area_um^2_median,area_um^2_mean,area_um^2_median,perimeter_um_mean,perimeter_um_median
0,0,1min_Fri_FOV1_decon.tif,Untransduced,Untransduced,Basal,1min,35.275554,64.691886,0.006451,0.005188,0.080183,0.027201,1.034718,0.587419
1,216,1min_Fri_FOV1_decon.tif,Untransduced,Untransduced,Cytosolic,1min,48.377303,60.571594,0.006451,0.005188,0.080183,0.027201,1.034718,0.587419
2,425,1min_Fri_FOV2_decon.tif,Untransduced,Untransduced,Basal,1min,29.070954,65.315825,0.008549,0.007110,0.106238,0.036199,1.112273,0.659071
3,566,1min_Fri_FOV2_decon.tif,Untransduced,Untransduced,Cytosolic,1min,37.777322,56.026598,0.008549,0.007110,0.106238,0.036199,1.112273,0.659071
4,692,1min_Fri_FOV3_decon.tif,Untransduced,Untransduced,Basal,1min,20.708930,83.686319,0.037439,0.028700,0.371468,0.109649,1.862846,1.042060


In [ ]:
# output_notebook()
# # un-comment the line below to output to an html file
# output_file('bokeh_COVID.html')

In [75]:
source = ColumnDataSource(df)
MESHRANGE = ['Untransduced', 'CAR']
factors = [
    ('Untransduced', 'Basal'),
    ('Untransduced', 'Cytosolic'),
    ('CAR', 'Basal'),
    ('CAR', 'Cytosolic'),
]
df['cats'] = df.apply(lambda x: (x['cell_type_dual'], x['mesh_type']), axis=1)
df['cats'].unique()

array([('Untransduced', 'Basal'), ('Untransduced', 'Cytosolic'),
       ('CAR', 'Basal'), ('CAR', 'Cytosolic')], dtype=object)

In [112]:
color_map = factor_cmap(field_name='mesh_type',palette=Pastel2[3][0:2], factors=df['mesh_type'].unique())

In [113]:
p2 = figure(
    height=200,
    width=400,
    x_range=FactorRange(*factors),
    tools='save'
)
p2.vbar(
    source=df,
    x='cats',
    top='cell_surface_area_um^2',
    color=color_map
    #height=0.6,
)

# p2.ygrid.grid_line_color = None
show(p2)

In [27]:
p = figure(plot_width=800, plot_height=300, x_range=MESHRANGE, 
           title="NAH")
p.circle(x='cell_type_dual', y=jitter('cell_surface_area_um^2', width=0.6),  source=source, alpha=0.3)
#p.vbar(x='cell_type_dual', top='cell_surface_area_um^2', width=0.6,  source=source, alpha=0.3)
p.x_range.range_padding = 0
p.ygrid.grid_line_color = None

show(p)

RuntimeError: Models must be owned by only a single document, UnionRenderers(id='1811', ...) is already in a doc

In [ ]:
# create a ColumnDataSource containing only the data I want to plot
# (Note: I only need to convert the pandas DataFrame to a ColumnDataSource because I want to manipulate it later in Javascript)
source = ColumnDataSource(
    data = dict(
        x = df['cell_type_dual'], 
        y = df['cell_surface_area_um^2'], # included for the tooltips

    )
)

# create a ColumnDataSource containing all the necessary data so that I can send it to javascript for filtering
allSource = ColumnDataSource(
    data = dict(
        Cell_surface_area = df['cell_surface_area_um^2'], 
        Mesh_density = df['mesh_density'],
        Hole_area_median = df['area_um^2_median'],
        Equivalent_diameter_area_median = df['equivalent_diameter_area_um^2_median'],
        Hole_perimeter_median = df['perimeter_um_median'],
    )
)


# define the tools you want to use
TOOLS = "pan,wheel_zoom,box_zoom,reset,save"

# define the tooltip
hover_tool = HoverTool(
    tooltips=[
        ( 'filename',   '@y{str}'),
        ( 'cell_type',  '@y{str}' ),
        ( 'mesh_type',  '@x{str}' ),
    ],
    # display a tooltip whenever the cursor is vertically in line with a glyph
    #mode = 'vline'
)

# create a new plot  
f = figure(tools = TOOLS, 
    width = 800, 
    height = 400, 
    x_range = [np.nanmin(usedf['Date_reported']), np.nanmax(usedf['Date_reported'])],
    y_range = [max(np.nanmin(usedf['New_cases_rolling']),0), np.nanmax(usedf['New_cases_rolling'])],
    x_axis_label = 'Date',
    y_axis_label = 'COVID-19 Count (' + str(rollingAve) + '-day rolling average)'
)
f.tools.append(hover_tool)

# fill the area
f.varea(x = 'x', y1 = 'y', y2 = 0,   
    source = source, 
    color = 'black', 
    alpha = 0.5
)

# draw the line
f.line('x', 'y',
    source = source, 
    color = 'black', 
    alpha = 1, 
    line_width = 2
)

# create the dropdown menu 
# (Note: Bokeh call's this select, because of the html nomenclature; there is also a difference Bokeh Dropdown)
select = Select(title = '', 
    value = country,
    options = df['Country'].unique().tolist(),
    width = 200,
    margin = (5, 5, 5, 80)
)

# create some radio buttons
radio = RadioButtonGroup(
    labels = ['Daily Cases', 'Daily Deaths', 'Cumulative Cases' ,'Cumulative Deaths'], 
    active = 0,
    width = 100,
    margin = (5, 5, 5, 80)
)

# Javascript code for the callback
callback = CustomJS(
    args = dict(
        source = source, 
        allSource = allSource,
        select = select,
        radio = radio,
        ranges = dict(
            x = f.x_range, 
            y = f.y_range
        ) 
    ),
    code = 
    """
        // get the value from the dropdown
        var country = select.value;

        //convert the value from the radio button to a key
        var key = null;
        if (radio.active == 0) key = 'New_cases_rolling';
        if (radio.active == 1) key = 'New_deaths_rolling';
        if (radio.active == 2) key = 'Cumulative_cases_rolling';
        if (radio.active == 3) key = 'Cumulative_deaths_rolling';      
        
        // filter the full data set to include only those from that country
        if (key){
            var x = allSource.data.Date_reported.filter(function(d,i){return allSource.data.Country[i] == country});
            var y = allSource.data[key].filter(function(d,i){return allSource.data.Country[i] == country});

            //update the data in the plot
            source.data.x = x;
            source.data.y = y;
            source.change.emit();

            //reset the axis limits
            //note that this ... syntax may not work on all browsers
            ranges.x.start = Math.min(...x); 
            ranges.x.end =  Math.max(...x);
            ranges.y.start = Math.max(Math.min(...y),0); 
            ranges.y.end =  Math.max(...y);
        }
        
    """
)

#attach the callback 
select.js_on_change('value', callback)
radio.js_on_click(callback)


show(column([
    Div(text = '<h1>Bokeh COVID-19 Data Explorer</h1>'), 
    select,
    radio,
    f]
))